In [1]:
import pandas as pd
import re
from ast import literal_eval
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '90'
formatted_attribute = 'width'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
curation_col = f'Q:{attribute}'
field1='Width [Nom]'
fields = [field1]

In [6]:
df_cur, df_custom=parameters(customer_id, formatted_attribute, field1)
df_custom=custom_field(df_custom,field1,formatted_attribute)

In [7]:
matchcust=df_custom[['external_id',curation_col]]
print(len(matchcust))
rounding(df_custom,curation_col)

KeyError: "['external_id'] not in index"

In [8]:
del df_cur['bucket_id']
del df_cur['product_id']
del df_cur['image_url']
del df_cur['value']

# General Regex

In [9]:
regex_pattern_length=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3Le?n?g?t?h?\b)|()'''
df_cur['matches_length'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'''(?i)((?<!up.to.)(?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}He?i?g?h?t?(?! Adjustable)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'''        
df_cur['matches_height'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
regex_pattern_width=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Wi?d?t?h?\b)|()'''
df_cur['matches_width'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'''(?i)((?<!\d)(?<!\w )(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Diam?e?t?e?r?\b)|()'''
df_cur['matches_diameter'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
# regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
# df_cur['matches_ft'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
# regex_pattern_blank=r'\d'                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# df_cur['new_matches'] = df_cur['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'''(?i)((?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)?\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?x\W?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)?\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?x?\W?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)?\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)?\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?)|((?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?)|()'''
df_cur['blank_product'] = df_cur['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_cur['blank_long'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

# Width

In [13]:
from natsort import natsorted
regex_pattern=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}(?:\'|\'\'|\"|\”|inc?h?|fe?e?t|foot)[\W]{0,3}(?:Width|W)\b)|()'''           
df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

# Filter out the SKUs that dont have numbers
pat_number=r'\d'
df_cur['new_matches'] = df_cur['matches'].apply(lambda x: re_extract(pat_number, str(x)))
lengthNew=df_cur[(df_cur['new_matches'].astype(str)!='[]')&(df_cur['matches'].astype(str)!='[]')]
del lengthNew['new_matches']

lengthNew['matches']=lengthNew['matches'].apply(lambda x: natsorted(x)).apply(lambda x: sorted(x))
lengthNew['matchez']=lengthNew['matches'].apply(lambda x: re.sub(r'''(?i)((?:\\?\"|\”|inc?h?|'')?\W{0,2}(?:Width|W))|(\\ in)|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'\\" W',' in',str(x))).apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"|�|•','',str(x))).apply(lambda x: re.sub(r'''(?i)(\\?(?:(?:(?<!')'(?!'))|(?:ft)|(?:foot)|(?:feet))\W?(Width|W))''',' ft',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))


curate(lengthNew, 'matchez', formatted_attribute, 'a-eghj-mo-su-z')
print(len(lengthNew))
matchesnew=lengthNew[['external_id',curation_col]]
matchesnew[curation_col].explode().value_counts()
rounding(lengthNew, 'matchez','a-eghj-mo-su-z')

0


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, url, custom_fields, matches_length, matches_height, matches_width, matches_diameter, blank_product, blank_long, matches, matchez, Q:width, rounding]
Index: []

In [14]:
x=df_cur[df_cur['matches'].astype(str)=='[]']
print(len(x[x['blank_long'].astype(str)!='[]']))
x[x['blank_long'].astype(str)!='[]'][0:500]

1367


buckets external_id  \
0                            Disposable Food Storage  5637365301   
1                                    Activity Tables  5637508326   
2                                    Activity Tables  5637508416   
3                                    Activity Tables  5637508417   
4                                    Activity Tables  5637508451   
5                                    Activity Tables  5637508452   
6                                    Activity Tables  5637576680   
7                                    Activity Tables  5637576682   
8                                    Correction Aids  5637305392   
9                                Fasteners & Magnets  5637227808   
10                               Fasteners & Magnets  5637227809   
11                               Fasteners & Magnets  5637219870   
12                               Fasteners & Magnets  5637246746   
13                               Fasteners & Magnets  5637246748   
14                               Fasteners & Magnets  5637246788   
15                               Fasteners & Magnets  5637246745   
16                               Fasteners & Magnets  5637284463   
17                               Fasteners & Magnets  5637287689   
18                               Fasteners & Magnets  5637302783   
19                               Fasteners & Magnets  5637302967   
20                               Fasteners & Magnets  5637302968   
21                               Fasteners & Magnets  5637302969   
22                               Fasteners & Magnets  5637328364   
23                               Fasteners & Magnets  5637366955   
24                               Fasteners & Magnets  5637366956   
25                               Fasteners & Magnets  5637366958   
26                               Fasteners & Magnets  5637366960   
27                               Fasteners & Magnets  5637366962   
28                               Fasteners & Magnets  5637366965   
29                               Fasteners & Magnets  5637366969   
30                               Fasteners & Magnets  5637366978   
31                               Fasteners & Magnets  5637366979   
32                               Fasteners & Magnets  5637366644   
33                               Fasteners & Magnets  5637366645   
34                               Fasteners & Magnets  5637394948   
35                               Fasteners & Magnets  5637394949   
36                               Fasteners & Magnets  5637394950   
37                               Fasteners & Magnets  5637394951   
38                               Fasteners & Magnets  5637394971   
39                               Fasteners & Magnets  5637394972   
40                               Fasteners & Magnets  5637417319   
41                               Fasteners & Magnets  5637417320   
42                               Fasteners & Magnets  5637437908   
43                               Fasteners & Magnets  5637145277   
44                               Fasteners & Magnets  5637169300   
45                               Fasteners & Magnets  5637169301   
46                               Fasteners & Magnets  5637169304   
47                               Fasteners & Magnets  5637169139   
48                               Fasteners & Magnets  5637490850   
49                               Fasteners & Magnets  5637490851   
50                               Fasteners & Magnets  5637493032   
51                               Fasteners & Magnets  5637499198   
52                               Fasteners & Magnets  5637499199   
53                               Fasteners & Magnets  5637496973   
54                               Fasteners & Magnets  5637504151   
55                               Fasteners & Magnets  5637504152   
56                               Fasteners & Magnets  5637504153   
57                               Fasteners & Magnets  5637504154   
58                               

# N/A Values

In [15]:
df_na=df_cur
regex_pattern_na=r'''(?i)((?<!Cord Measures )(?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)['"”].{0,2}(?:[LWHD]|diam)?)|(measure)|()'''
df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))

df_na=df_na[(df_na['matches_height'].astype(str)=='[]')&(df_na['na_matches'].astype(str)=='[]')&(df_na['blank_product'].astype(str)=='[]')&(df_na['blank_long'].astype(str)=='[]')]
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
print(len(matchesna))
# matchesna

0


In [22]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnew))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcust))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))

matchesnew
matchcust
matchesna


In [ ]:
stop

# send to the folder for upload

In [23]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesnew)

In [24]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchcust)

In [25]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesna)